# Ecosystem services widgets  
This contains two widgets with independent data:
1. Above/below ground organic carbon
2. Commercial fishing restoration

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import fiona

## Above/below ground organic carbon data

In [8]:
gdb_file =f'../../../../data/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration_20190411.gdb'
all_data = gpd.read_file(gdb_file, driver='FileGDB', layer=0)
all_data.head()

,Country,Tot_Restor,Restor_pct,Rest_Area_Loss,Area_loss_pct,Rest_Area_Dgrd,Area_dgrd_pct,Total_2016,Mean_Score,SOC,AGB,People,Fish_Score,Fish_Score_Inv,Shape_Length,Shape_Area,Loss_Driver,geometry
0,American Samoa,0.00000,0,0.00,0,0.00,0,18.74,0,0.000000,0.000000,0,109000000.0,200000.0,5.350212e+04,1.638363e+07,No Data,"MULTIPOLYGON (((-19011686.866 -1613074.241, -1..."
1,Angola,580.11556,4,671.43,5,68.92,0,13286.05,62,220274.238512,30809.263129,3800,-1.0,-1.0,1.383794e+06,2.785231e+10,NPC,"MULTIPOLYGON (((1550210.491 -1230695.594, 1549..."
2,Anguilla,0.08128,4,1.27,59,0.00,0,0.87,24,407.670000,6.524752,0,1000000.0,2000000.0,2.097797e+04,1.419573e+06,No Data,"MULTIPOLYGON (((-7025249.376 2058315.576, -702..."
3,Antigua and Barbuda,14.15571,2,20.19,2,0.00,0,886.30,22,9031.556968,1078.042000,500,8000000.0,13000000.0,1.888816e+05,1.361710e+09,No Data,"MULTIPOLYGON (((-6873186.951 1921601.093, -688..."
4,Aruba,19.38207,36,20.55,38,0.00,0,33.79,46,9816.581150,1374.874165,0,71000000.0,119000000.0,3.798229e+04,2.166772e+07,No Data,"MULTIPOLYGON (((-7781925.061 1393950.107, -778..."


In [3]:
carbon_data = all_data[['Country', 'SOC', 'AGB']].copy()
carbon_data

,Country,SOC,AGB
0,American Samoa,0.000000e+00,0.000000e+00
1,Angola,2.202742e+05,3.080926e+04
2,Anguilla,4.076700e+02,6.524752e+00
3,Antigua and Barbuda,9.031557e+03,1.078042e+03
4,Aruba,9.816581e+03,1.374874e+03
...,...,...,...
103,Vanuatu,5.545036e+03,8.574929e+02
104,Venezuela,4.366034e+06,9.770106e+05
105,Vietnam,5.100087e+06,1.311131e+06
106,"Virgin Islands, U.S.",1.609178e+03,2.919104e+02


In [4]:
carbon_long = pd.melt(carbon_data, id_vars='Country')
carbon_long.rename(columns={'variable':'indicator'}, inplace=True)
carbon_long

,Country,indicator,value
0,American Samoa,SOC,0.000000e+00
1,Angola,SOC,2.202742e+05
2,Anguilla,SOC,4.076700e+02
3,Antigua and Barbuda,SOC,9.031557e+03
4,Aruba,SOC,9.816581e+03
...,...,...,...
211,Vanuatu,AGB,8.574929e+02
212,Venezuela,AGB,9.770106e+05
213,Vietnam,AGB,1.311131e+06
214,"Virgin Islands, U.S.",AGB,2.919104e+02


In [6]:
# Import locations to get staging ids
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country']
locations.head()

,name,iso,type,area_m2,wdpaid,globalid,perimeter_m,location_idn,coast_length_m,geometry
82,Qatar,QAT,country,3.880224,NaN,{AF97ABE2-6405-4438-A7ED-1494A43DA379},8.392644,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd,1345769.96,"MULTIPOLYGON (((50.73769 24.93464, 50.73779 24..."
89,Mayotte,MYT,country,5.611808,NaN,{57E86B5B-7EF0-4754-A8D4-A9DC3212D421},10.086238,0750953f-4af9-549b-aeea-329663249a56,291036.71,"POLYGON ((46.63483 -12.96039, 46.63197 -12.969..."
118,Vietnam,VNM,country,90.156489,NaN,{B2A84FBB-34CD-4A51-9463-B9DB2DB62A10},81.714911,09a1ab14-11ad-56ec-8acb-a149e5697abd,9005760.08,"MULTIPOLYGON (((104.31952 10.36051, 104.31975 ..."
132,Grenada,GRD,country,2.154728,NaN,{F8753179-5FFA-4D9E-8AD9-083F31C48528},6.743601,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e,260664.47,"MULTIPOLYGON (((-61.91525 11.37330, -61.91813 ..."
149,India,IND,country,473.029671,NaN,{A4A6CE4D-8D03-4246-9A2F-BD9811232115},211.564078,0c07ca53-7b17-5650-a2c6-0cc27249a4bd,16917891.22,"MULTIPOLYGON (((79.52922 9.38411, 79.52921 9.3..."


In [7]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


In [9]:
# Import gadm file to retrieve ISO codes
gadm = gpd.read_file('../../../../data/gadm36_level0_original/gadm36_level0_original.shp')
gadm.head()

,GID_0,NAME_0,AREA_KM2,MOL_ID,Shape_Leng,Shape_Area,geometry
0,ABW,Aruba,1.819384e+02,1,0.963634,0.015131,"POLYGON ((-69.97820 12.46986, -69.97847 12.469..."
1,AFG,Afghanistan,6.438575e+05,2,57.103371,62.749594,"POLYGON ((68.52644 31.75435, 68.53852 31.75457..."
2,AGO,Angola,1.247422e+06,3,73.796528,103.818655,"MULTIPOLYGON (((11.73347 -16.67255, 11.73347 -..."
3,AIA,Anguilla,8.330331e+01,4,1.318321,0.007116,"MULTIPOLYGON (((-63.42375 18.58903, -63.42375 ..."
4,ALA,Åland,1.506261e+03,5,42.232199,0.243769,"MULTIPOLYGON (((21.32195 59.74986, 21.32195 59..."


In [23]:
#Prepare locations data
rp = all_data[['Country']]

isocodes = gadm[['GID_0', 'NAME_0']]

rp = pd.merge(rp, isocodes, how='left', left_on='Country', right_on='NAME_0')
rp.rename(columns={'GID_0':'iso'}, inplace=True)

rp.head()


,Country,iso,NAME_0
0,American Samoa,ASM,American Samoa
1,Angola,AGO,Angola
2,Anguilla,AIA,Anguilla
3,Antigua and Barbuda,ATG,Antigua and Barbuda
4,Aruba,ABW,Aruba


In [24]:
rp[rp.iso.isnull()]

,Country,iso,NAME_0
12,Bonaire,NaN,NaN
30,East Timor,NaN,NaN
83,Sao Tome and Principe,NaN,NaN


In [27]:
rp.loc[rp.Country == 'East Timor', 'iso'] = 'TLS'
rp.loc[rp.Country == 'Bonaire', 'iso'] = 'BQ'
rp.loc[rp.Country == 'Sao Tome and Principe', 'iso'] = 'STP'
rp[rp.iso.isnull()]


,Country,iso,NAME_0


In [28]:
rp.drop(['NAME_0'], axis=1, inplace=True)
rp.head()

,Country,iso
0,American Samoa,ASM
1,Angola,AGO
2,Anguilla,AIA
3,Antigua and Barbuda,ATG
4,Aruba,ABW


In [29]:
#Add staging locations id codes
final_locations = pd.merge(rp, locations[['iso', 'location_idn']], how='left', on='iso')
final_locations

,Country,iso,location_idn
0,American Samoa,ASM,404d005a-797d-5509-91eb-e17ed1069ed6
1,Angola,AGO,27ceab8c-946e-5286-a06f-8bd98ec81f77
2,Anguilla,AIA,1ce4c2e5-8456-5db8-8e34-8bfe86083790
3,Antigua and Barbuda,ATG,7c8d9de5-4c1a-5ed4-838c-05906eaed3f7
4,Aruba,ABW,5d1bd36b-f200-593d-972a-bf69abdd912f
...,...,...,...
103,Vanuatu,VUT,88ede091-9662-564b-95af-72a8b4cbda4b
104,Venezuela,VEN,a22a14b8-b4dd-55e6-af76-87f1bf68af2a
105,Vietnam,VNM,09a1ab14-11ad-56ec-8acb-a149e5697abd
106,"Virgin Islands, U.S.",VIR,3fb957bc-db23-5b2e-8f5d-d021133b9414


In [30]:
final_locations = pd.merge(final_locations, api_locs, how='left', on='location_idn')
final_locations.head()

,Country,iso,location_idn,id
0,American Samoa,ASM,404d005a-797d-5509-91eb-e17ed1069ed6,2346.0
1,Angola,AGO,27ceab8c-946e-5286-a06f-8bd98ec81f77,2029.0
2,Anguilla,AIA,1ce4c2e5-8456-5db8-8e34-8bfe86083790,1915.0
3,Antigua and Barbuda,ATG,7c8d9de5-4c1a-5ed4-838c-05906eaed3f7,3095.0
4,Aruba,ABW,5d1bd36b-f200-593d-972a-bf69abdd912f,2707.0


In [31]:
df_final = pd.merge(final_locations, carbon_long, on='Country', how='right')
df_final.rename(columns={'id':'location_id'}, inplace=True)
df_final

,Country,iso,location_idn,location_id,indicator,value
0,American Samoa,ASM,404d005a-797d-5509-91eb-e17ed1069ed6,2346.0,SOC,0.000000e+00
1,Angola,AGO,27ceab8c-946e-5286-a06f-8bd98ec81f77,2029.0,SOC,2.202742e+05
2,Anguilla,AIA,1ce4c2e5-8456-5db8-8e34-8bfe86083790,1915.0,SOC,4.076700e+02
3,Antigua and Barbuda,ATG,7c8d9de5-4c1a-5ed4-838c-05906eaed3f7,3095.0,SOC,9.031557e+03
4,Aruba,ABW,5d1bd36b-f200-593d-972a-bf69abdd912f,2707.0,SOC,9.816581e+03
...,...,...,...,...,...,...
211,Vanuatu,VUT,88ede091-9662-564b-95af-72a8b4cbda4b,3248.0,AGB,8.574929e+02
212,Venezuela,VEN,a22a14b8-b4dd-55e6-af76-87f1bf68af2a,3582.0,AGB,9.770106e+05
213,Vietnam,VNM,09a1ab14-11ad-56ec-8acb-a149e5697abd,1681.0,AGB,1.311131e+06
214,"Virgin Islands, U.S.",VIR,3fb957bc-db23-5b2e-8f5d-d021133b9414,2339.0,AGB,2.919104e+02


In [33]:
df_final[~df_final['location_id'].isna()][['location_id', 'indicator', 'value']].to_csv('../../../../data/UPDATED_ecosystem_services_carbon_data.csv', index=False)